## COMPARE STOCKS BEFORE INVESTING 

In [1]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
pd.options.display.float_format = "{:,.4f}".format
import datetime 
from datetime import date

In [9]:
def CompareStocks(tickers,startTime=datetime.date.today()-datetime.timedelta(365*2), endTime=datetime.date.today()):
    # pull price data from yahoo -- (list(tickers.keys())) = ['Stock A key','Stock B key']
    prices = pdr.DataReader(list(tickers.keys()), "yahoo", startTime, endTime)[
        "Adj Close"
    ]
    
    prices = prices.rename(columns=tickers)
    returns = np.log(prices) - np.log(prices.shift(1))
    returns = returns.iloc[1:, 0:]
    
    # pull data into separate DataFrame,for calculating our highLow metric
    # highLow Metric is VolTest1
    currYear = prices.loc[
        date.today() - datetime.timedelta(365) : date.today() 
    ]
    
    highLow = (currYear.max() - currYear.min()) / prices.iloc[-1]
    highLow = pd.DataFrame(highLow, columns=["VolTest1"])
    
    # Moving average volatility Metric, is VolTest2
    MA = pd.DataFrame(
        ((abs(prices - prices.rolling(50).mean())) / prices).mean(),
        columns=["VolTest2"],
    )
    
    investments = pd.merge(highLow, MA, on="Symbols")
    investments = pd.merge(
        investments,
        pd.DataFrame(returns.std(), columns=["StandardDeviation"]),
        on="Symbols",
    )
    
    investments = pd.merge(
        investments,
        pd.DataFrame(100 * returns.mean(), columns=["Daily Return %"]),
        on="Symbols",
    )
    
    return investments.round(4)

In [10]:
CompareStocks({"^GSPC": "Stock A", "^RUT": "Stock B"},)
#Choose correct Tickers as many as possible to compare stocks
#In this eg. ^GSPC is ticker for S&P500 while ^RUT is ticker for Russell2000

,VolTest1,VolTest2,StandardDeviation,Daily Return %
Symbols,,,,
Stock A,0.3081,0.0315,0.0112,0.0325
Stock B,0.4806,0.0438,0.0153,0.0289
